In [1]:
%cd ..

/home/Public/Megatron-LM


In [2]:
%%time

from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer

tokenizer = make_tokenizer(
    SentencePieceTokenizer,
    None,
    model_path='./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'
)

# tokenizer.command_tokens
for tok in tokenizer.command_name_map.values():
    print(f'{tok.Id}: {tok.token}')

0: <pad>
1: <eos>
2: <bos>
3: <unk>
4: <sep>
5: <L2R>
6: <ENC>
7: <MASK>
CPU times: user 1.27 s, sys: 173 ms, total: 1.44 s
Wall time: 1.48 s


In [3]:
import math
import random
from itertools import chain
from more_itertools import windowed

from forumqa.data import ForumQaDataset, split_sentences

In [4]:
files = '/home/liuxy/Public/Megatron-LM/data/xinliqa/5.json'
ds = ForumQaDataset(files, tokenizer, repeat_times=100)


In [5]:
len(ds)

500

In [7]:
def print_item(idx):
    print(f'[{idx}]:')
    ids = ds[idx]
    print(len(ids))
    while ids[-1] < 8:
        del ids[-1]
    print(tokenizer.DecodeIds(ids))
    print()

In [17]:
for i in range(5):
    %time print_item(i)

[0]:
1025
22岁,做直销 <sep> 父母都很支持,可是我性格内向,而且不喜欢说话,圈内有很多成功了的大神,我没自信能不能成功,直销靠谱不 <sep> 职业管理 <sep>  <sep>  <bos> 你好,看来你对自己的认识还挺清晰的,让我想到心理学荣格大师的话,一个人向外看他在梦中,向内看他会醒来,所以认清自己真的对于减少心理冲突很有帮助。你现在感觉自己性格内向到底是否适合直销感觉怀疑,那么首先问问你自己你的兴趣爱好是什么呢,是否适合干这些工作,你想要会怎么样的生活和何种工作方式,那种方式会让你更自主呢,在就你所说从增加自信开始,如何增加自信呢,一次次完成所定力的目标并且进行自我奖赏慢慢自信就会提高拥有自主决策的能力。

CPU times: user 2.19 ms, sys: 0 ns, total: 2.19 ms
Wall time: 1.59 ms
[1]:
1025
真理都是相通的? <sep> 从洞穴出来看见理性之光的形而上人开始拼凑真理的碎片,到达二点零的人拼凑起来啦一部分碎片形成自己一套看世界的体系。 这时候的人都会用自己的一套概念体系去描述他研究得那一部分真理,可是你会发现他们说得都是一回事——真理都是相通的只是表述不同罢啦 <sep> 心理咨询 <sep>  <sep> 素质教育里能力培养(把知识学活,从客观世界去提取概念架构成魔方并学以致用以满足生物人的趋利避害)这一步我得抓紧时间完成加油加油加油加油加油加油加油加油加油加油加油加油加油加油加油加油加油加油加油加油

CPU times: user 1.41 ms, sys: 18 µs, total: 1.42 ms
Wall time: 1.26 ms
[2]:
1025
爸妈的话让我很难过 <sep> 我26岁,生活在一个小城市。爸妈周围同事朋友的孩子大部分都结婚了,只有我还没对象。最近老爸又收到一个请帖,然后对我说,你也赶紧给我结婚!你都让我感觉到丢人!我当时就火了,和父母大吵一架。虽然我当时脾气很差,但因为逼婚我心里一直压力很大,我知道父母着急,可是我也没法马上解决结婚的问题。我最介怀的是"丢人"二字,即使爸妈道歉说那是无心的,可无心的话恰恰是心里话,我没有怨他们,只是觉得我很悲哀,用他们的话来说是"没本事自己去找个对象,还不乐意别人给你相亲"。努力读书,工作,每天下班